<font size="6">Сегментация и детектирование</font>

# Задачи компьютерного зрения

В ходе предыдущих занятий мы подробно рассмотрели задачу классификации изображений.

Но порой недостаточно знать, что на изображении есть объект определенного класса. Важно, где именно расположен объект. В ряде случаев нужно знать еще и точные границы объекта. Например, если речь идет о рентгеновском снимке или изображении клеток ткани, полученном с микроскопа.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/classification_semantic_segmentation.png" width="600">

Определение того, какие фрагменты изображения принадлежат объектам определенных классов — это задача **сегментации** (segmentation).

Если нас интересуют не индивидуальные объекты, а только тип (класс) объекта, которым занят конкретный пиксель (как в случае с клетками под микроскопом), то говорят о **семантической сегментации** (semantic segmentation).

Если нас интересуют конкретные объекты, и при этом достаточно знать только область, в которой объект локализован, то это задача **детектирования** (Detection)

В качестве примера такой задачи можно рассмотреть подсчет количество китов на спутниковом снимке.


Если же важны и индивидуальные объекты, и их точные границы, то это уже задача **Instance segmentation** Например для  автопилота важно не просто что перед ним несколько автомобилей, а важно отличить где именно находится ближний а где дальний.

## Dataset COCO — Common Objects in Context

Прежде чем говорить о способах решения этих задач, надо разобраться с форматами входных данных. Сделаем это на примере датасета [COCO](https://cocodataset.org/).

Один из наиболее популярных датасатов, содержащий данные для сегментации и детектирования. Он содержит более трёхсот тысяч изображений, большая часть из которых размечена и содержит следующую информацию:
- Категории
- Маски
- Ограничивающие боксы (*bounding boxes*)
- Описания (*captions*)
- Ключевые точки (*keypoints*)
- И многое другое

Формат разметки изображений, использованный в этом датасете, нередко используется и в других наборах данных. Как правило, он упоминается просто как "COCO format".

Загрузим датасет.

In [ ]:
from IPython.display import clear_output


# !wget -N "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
!wget -N "https://edunet.kea.su/repo/EduNet-web_dependencies/L12/annotations_trainval2017.zip"
!unzip -n annotations_trainval2017.zip

clear_output()

Для работы с датасетом используется пакет `pycocotools`.

[Подробнее о том, как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

In [ ]:
from pycocotools.coco import COCO

coco = COCO("annotations/instances_val2017.json")

Рассмотрим формат аннотаций на примере одной записи.

In [ ]:
catIds = coco.getCatIds(catNms=["cat"])  # cat's IDs
print("class ID(cat) =", catIds)

imgIds = coco.getImgIds(catIds=catIds)  # Filtering dataset by tag
print("All images: %i" % len(imgIds))

Рассмотрим метаданные.

In [ ]:
img_list = coco.loadImgs(imgIds[0])  # 1 example
img_metadata = img_list[0]
img_metadata

Посмотрим на изображение.

In [ ]:
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO


def coco2pil(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


I = coco2pil(img_metadata["coco_url"])
plt.axis("off")
plt.imshow(I)
plt.show()

### Категории в COCO

Давайте посмотрим на категории в датасете. 

In [ ]:
cats = coco.loadCats(coco.getCatIds())  # loading categories
num2cat = {}
print("COCO categories: ")

iterator = iter(cats)
cat = next(iterator)
for i in range(0, 91):
    if i == cat["id"]:
        num2cat[cat["id"]] = cat["name"]
        name = cat["name"]
        if i < 90:
            cat = next(iterator)
    else:
        name = "---"

    print(f"{i:2}. {name:20}", end="")

    if not i % 6:
        print("\n")

Категория **0** используется для обозначения класса фона. Некоторые номера категорий не заняты ...

Также существуют надкатегории. 

In [ ]:
print(f"cats[2]: {cats[2]}")
print(f"cats[3]: {cats[3]}")

nms = set([cat["supercategory"] for cat in cats])
print("COCO supercategories: \n{}".format("\t".join(nms)))

### Разметка данных

Помимо метаданных, нам доступна разметка ([подробнее о разметке](https://cocodataset.org/#format-data)). Давайте её загрузим и отобразим.

In [ ]:
annIds = coco.getAnnIds(imgIds=img_metadata["id"])
anns = coco.loadAnns(annIds)

plt.imshow(I)
plt.axis("off")
coco.showAnns(anns)
plt.show()

На изображении можно увидеть разметку пикселей изображения по классам. То есть, пиксели из объектов, относящихся к интересующим классам, приписываются к классу этого объекта. К примеру, можно увидеть объекты двух классов: "cat" и "keyboard". 

Давайте теперь посмотрим, из чего состоит разметка.

In [ ]:
def dump_anns(anns):
    for i, a in enumerate(anns):
        print(f"\n#{i}")
        for k in a.keys():
            if k == "category_id" and num2cat.get(a[k], None):
                print(k, ": ", a[k], num2cat[a[k]])  # Show cat. name
            else:
                print(k, ": ", a[k])


dump_anns(anns)

Заметим, что аннотация изображения может состоять из описаний нескольких объектов, каждое из которых содержит следующую информацию:
* `segmentation` — последовательность пар чисел ($x$, $y$), координат каждой из вершин "оболочки" объекта;
* `area` — площадь объекта;
* `iscrowd` — несколько объектов, например толпа людей, в этом случае информация о границах объекта (маска) храниться в [RLE](https://en.wikipedia.org/wiki/Run-length_encoding) формате;
* `image_id` — идентификатор изображения, к которому принадлежит описываемый объект;
* `bbox` — *будет рассмотрен далее в ходе лекции*;
* `category_id` — идентификатор категории, к которой относится данный объект;
* `id` — идентификатор самого объекта.

Попробуем посмотреть на пример, в котором `iscrowd = True` .

In [ ]:
catIds = coco.getCatIds(catNms=["people"])
annIds = coco.getAnnIds(catIds=catIds, iscrowd=True)
anns = coco.loadAnns(annIds[0:1])

dump_anns(anns)
img = coco.loadImgs(anns[0]["image_id"])[0]
I = coco2pil(img["coco_url"])
plt.imshow(I)
coco.showAnns(anns)  # People in the stands
seg = anns[0]["segmentation"]
print("Counts", len(seg["counts"]))
print("Size", seg["size"])
plt.axis("off")
plt.show()

Что такое [run-length encoding (RLE)](https://en.wikipedia.org/wiki/Run-length_encoding)

[Видео-разбор](https://www.youtube.com/watch?v=h6s61a_pqfM)

Используя методы из `pycocotools`, можно  преобразовать набор вершин "оболочки" сегментируемого объекта в более удобный, но менее компактный вид — маску объекта.

In [ ]:
import numpy as np

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
msk = np.zeros(seg["size"])

fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))


i = 0
for row in range(4):
    for col in range(4):
        ann = anns[i]
        msk = coco.annToMask(ann)
        ax[row, col].imshow(msk, cmap="gray")
        ax[row, col].set_title(num2cat[anns[i]["category_id"]])
        ax[row, col].axis("off")
        i += 1

plt.show()

В некоторых случаях попиксельная разметка изображения может быть избыточной. К примеру, если необходимо посчитать количество человек на изображении, то достаточно просто каким-то образом промаркировать каждого из них, после чего посчитать количество наших "отметок". Одним из вариантов маркировки является "обведение" объекта рамкой (`bounding box`), внутри которой он находится. Такая информация об объектах также сохранена в аннотациях формата COCO.

In [ ]:
from PIL import Image, ImageDraw

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
draw = ImageDraw.Draw(I)

colors = {1: "white", 40: "lime"}  # person - white, glove - lime
for ann in anns:
    x, y, width, heigth = ann["bbox"]  # bounding box here
    color = colors.get(ann["category_id"], None)
    if color:
        draw.rectangle((x, y, x + width, y + heigth), outline=color, width=2)
plt.imshow(I)
plt.show()

# Семантическая сегментация (*Semantic segmentation*)

*Сегментация изображения — задача поиска групп пикселей, каждая из которых характеризует один смысловой объект.*

Технически это выглядит так. Есть набор изображений:


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_1.png" width="300">

Для каждого изображения есть маска W x H:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_2.png" width="300">

Маска задает класс объекта для каждого пикселя:
[ x,y - > class_num ]

Набор таких изображений с масками — это и есть наш датасет, на нем мы учимся.

На вход модель получает новое изображение:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_3.png" width="600">

и должна предсказать метку класса для каждого пикселя (маску).

[ x,y - > class_num ] 

Получим такую маску из COCO:

In [ ]:
from IPython.display import clear_output


# !wget -N "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
!wget -N "https://edunet.kea.su/repo/EduNet-web_dependencies/L12/annotations_trainval2017.zip"
!unzip -n annotations_trainval2017.zip

clear_output()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import BytesIO
from PIL import Image, ImageDraw
from pycocotools.coco import COCO


def coco2pil(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


coco = COCO("annotations/instances_val2017.json")
clear_output()

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
img = coco.loadImgs(anns[0]["image_id"])[0]
I = coco2pil(img["coco_url"])

semantic_seg_person_mask = np.zeros(I.size[::-1], dtype=bool)  # WxH -> HxW

for ann in anns:
    msk = coco.annToMask(ann)  # HxW
    if ann["category_id"] == 1 and not ann["iscrowd"]:  # single person:
        # semantic_seg_person_mask = msk | semantic_seg_person_mask  # union
        semantic_seg_person_mask += msk.astype(bool)

semantic_seg_person_mask = semantic_seg_person_mask > 0  # binarize
plt.imshow(semantic_seg_person_mask, cmap="gray")
plt.show()

## Способы предсказания класса для каждого пикселя

Давайте подумаем о том, как такую задачу можно решить.

Из самой постановки задачи видно, что это задача классификации. Только не 
всего изображения, а каждого пикселя.

**a) Наивный**




Простейшим вариантом решения является использование так называемого "скользящего окна" — последовательное рассмотрение фрагментов изображения. В данном случае интересующими фрагментами будут небольшие зоны, окружающие каждый из пикселей изображения. К каждому из таких фрагментов применяется свёрточная нейронная сеть, предсказывающая, к какому классу относится центральный пиксель.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/naive_way_predict_pixel_class.png" width="600">

**б) Разумный**

Понятно, что запускать классификатор для каждого пикселя абсолютно неэффективно, так как для одного изображения потребуется $H*W$ запусков.

Можно пойти другим путем: получить карту признаков и по ней делать предсказание для всех пикселей разом.

Для этого потребуется поменять привычную нам архитектуру сверточной сети следующим образом:

* убрать слои, уменьшающие пространственные размеры;

* убрать линейный слой в конце, заменив его сверточным.





<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/reasonable_way_predict_pixel_class.png" width="700">

Теперь пространственные размеры выхода (W,H) будут равны ширине и высоте исходного изображения.

Количество выходных каналов будет равно количеству классов, которые мы учимся предсказывать. 

Тогда можно использовать значения каждой из карт активаций на выходе последнего слоя сети как ненормированное значение вероятности принадлежности (core) каждого из пикселей к тому или иному классу. 

То есть номер канала с наибольшим значением будет соответствовать классу объекта, который изображает данный пиксель.

In [ ]:
import torch

last_layer_output = torch.randn((3, 32, 32))  # class_num, W,H
print("Output of last layer shape", last_layer_output.shape)  # activation slice
mask = torch.argmax(last_layer_output, dim=0)  # class_nums prediction
print("One class mask shape", mask.shape)
print("Predictions for all classes \n", mask[:5, :5])

Чтобы на выходе сети получить количество каналов равное количеству классов, используется свертка 1x1.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/1x1_kernel_size_fully_connected_layer.png" width="500">

В лекции про сверточные сети мы говорили о том, что свертку 1x1 можно рассматривать как аналог полносвязного слоя. Именно так она тут и работает.

**Проблемы:**
- Что рецептивное поле нейронов на последних слоях было сопоставимо с размером изображения, требуется много сверточных слоев ($L$ раз свёртка $3\times3$ $\to$ рецептивное поле $(1+2L)$);
- Свертки медленно работают на полноразмерных картах активации.

**в) Эффективный**

Используем стандартную сверточную сеть, но полносвязанные слои заменим на сверточные.

## Fully Convolutional Networks

[Fully Convolutional Networks for Semantic Segmentation](https://arxiv.org/abs/1605.06211)


Сокращенно FCN. Для того, чтобы не было путаницы с Fully Connected Network, последние именуют MLP (Multi Layer Perceptron).



За основу берется обычная сверточная сеть для калассификации:







<center><img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/fcn_backbone.png" width="500"></center>

<center><p><em>Source: <a href="https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1"> Review: FCN — Fully Convolutional Network (Semantic Segmentation) </a></p> </em></center>


Такую сеть можно построить, взяв за основу другую сверточную архитектуру (*backbone*) , например `ResNet50` или `VGG16`.

<center><img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/fcn_changes.png" width="500"></center>

<center><p><em>Source: <a href="https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1"> Review: FCN — Fully Convolutional Network (Semantic Segmentation) </a></p> </em></center>

И затем заменить полносвязные слои на свертки.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/fully_convolution_network_scheme.png" width="500"></center>

<center><p><em>Source: <a href="https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1"> Review: FCN — Fully Convolutional Network (Semantic Segmentation) </a></p> </em></center>

В конце добавить `upsample` слой до нужных нам размеров. 

На вход такая модель может получать изображение произвольного размера. 
Так как для задач сегментации изменение размеров входного изображения приводит к потере важной информации о границах.

## Разжимающий слой

Как реализовать декодировщик?

### Интерполяция при увеличении разрешения 

Вспомним, как повышают разрешение для обычных изображений, а уже затем перейдем к картам признаков.

Допустим, требуется увеличить изображение размером 2x2 до размера 4x4.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/upsample.png" width="300">

<img src ="https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Comparison_of_1D_and_2D_interpolation.svg/1920px-Comparison_of_1D_and_2D_interpolation.svg.png" width="800">

[Bilinear_interpolation](https://en.wikipedia.org/wiki/Bilinear_interpolation)

Если для интерполяции используются значения четырех соседних пикселей, то такая интерполяция называется билинейной. В качестве интерполированного значения используется взвешенное среднее этих четырёх пикселей. 

In [ ]:
import seaborn as sns


def img_to_heatmap(img, ax, title):  # Magik method to show img as heatmap
    ax.axis("off")
    ax.set_title(title)
    array = np.array(img)
    array = array[None, None, :]
    sns.heatmap(array[0][0], annot=True, ax=ax, lw=1, cbar=False)


# Fake image
raw = np.array([[1, 3, 0, 1], [3, 3, 3, 7], [8, 1, 8, 7], [6, 1, 1, 1]], dtype=np.uint8)
pil = Image.fromarray(raw)

interp_nn = pil.resize((8, 8), resample=Image.NEAREST)
interp_bl = pil.resize((8, 8), resample=Image.BILINEAR)

# plot result
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)
img_to_heatmap(raw, ax[0], "Raster dataset")
img_to_heatmap(interp_nn, ax[1], "Nearest neighbor interpolation")
img_to_heatmap(interp_bl, ax[2], "Bilinear interpolation")
plt.show()

[Билинейная интерполяция](https://en.wikipedia.org/wiki/Bilinear_interpolation) позволяет избавиться от резких границ, которые возникают при увеличении методом ближайшего соседа.  Существуют и другие виды интерполяции, использующие большее количество соседних пикселей.





### Upsample в PyTorch

К чему был этот разговор об увеличении картинок?

Оказывается, для увеличения пространственного разрешения карт признаков (feature maps) можно применять те же методы, что и для изображений.

Для увеличения пространственного разрешения карт признаков (карт активаций), в PyTorch используется класс `Upsample`. В нём доступны все упомянутые методы интерполяции, а также трилинейная интерполяция — аналог билинейной интерполяции, используемый для работы с трёхмерными пространственными данными (к примеру, видео). 


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/upsample_pytorch.png" width="1000">

<center><p><em>Source: <a href="https://pytorch.org/docs/stable/generated/torch.nn.Upsample.html"> PyTorch nn.Upsample </a></p> </em></center>

[[doc] nn.functional.interpolate](https://pytorch.org/docs/stable/generated/torch.nn.functional.interpolate.html?highlight=interp#torch.nn.functional.interpolate)

Таким образом, мы можем использовать `Upsample` внутри нашего разжимающего блока.

Загрузим изображение:

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_1.png -O cat.png

In [ ]:
from torch import nn
import torchvision.transforms.functional as TF


def upsample(pil, ax, mode="nearest"):
    tensor = TF.to_tensor(pil)
    # Create upsample instance

    if mode == "nearest":
        upsampler = nn.Upsample(scale_factor=2, mode=mode)
    else:
        upsampler = nn.Upsample(scale_factor=2, mode=mode, align_corners=True)

    tensor_128 = upsampler(tensor.unsqueeze(0))  # add batch dimension
    # Convert tensor to pillow
    img_128 = tensor_128.squeeze()
    img_128_pil = TF.to_pil_image(img_128.clamp(min=0, max=1))
    ax.imshow(img_128_pil)
    ax.set_title(mode)


# Load and show image in Pillow format
pic = Image.open("cat.png")
pil_64 = pic.resize((64, 64))
fig, ax = plt.subplots(ncols=4, figsize=(15, 5))
ax[0].imshow(pil_64)
ax[0].set_title("Raw")

# Upsample with Pytorch
upsample(pil_64, mode="nearest", ax=ax[1])
upsample(pil_64, mode="bilinear", ax=ax[2])
upsample(pil_64, mode="bicubic", ax=ax[3])
plt.show()

Обратите внимание, что в данном случае каждое из пространственных измерений изображения увеличилось в 2 раза, но при необходимости возможно использовать увеличение в иное, в том числе не целое, количество раз, используя параметр `scale_factor`

Слои Upsample обычно комбинируют вместе со сверточными, это рекомендованный способ увеличения пространственных размеров карт признаков.

In [ ]:
model = nn.Sequential(
    nn.Upsample(scale_factor=2), nn.Conv2d(3, 16, kernel_size=3, padding=1), nn.ReLU()
)

dummy_input = torch.randn((0, 3, 32, 32))
out = model(dummy_input)
print(out.shape)

### Другие способы "разжать" карту признаков

#### MaxUnpooling

Помимо свёртки, на этапе снижения размерности также используются слои pooling'а. Наиболее популярным вариантом является MaxPooling, сохраняющий значение только наибольшего элемента внутри сегмента. Для того, чтобы обратить данную операцию субдискретизации, был предложен MaxUnpooling слой.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/maxunpooling.png" width="650">

Данный слой требует сохранения индексов максимальных элементов внутри сегментов — при обратной операции максимальное значение помещается на место, в котором был максимальный элемент сегмента до соответствующей субдискретизации. Соответственно, каждому слою MaxUnpooling должен соответствовать слой MaxPooling, что визуально можно представить следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/downsample_and_upsample_layers.png" width="650">

[Документация к MaxPool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

[Документация к MaxUnpool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxUnpool2d.html?highlight=unpooling)

In [ ]:
import requests
from io import BytesIO

torch.use_deterministic_algorithms(False, warn_only=False)


def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


def tensor_show(tensor, title="", ax=ax):
    img = TF.to_pil_image(tensor.squeeze()).convert("RGB")
    ax.set_title(title + str(img.size))
    ax.imshow(img)


pool = nn.MaxPool2d(
    kernel_size=2, return_indices=True
)  # False by default(get indexes to upsample)
unpool = nn.MaxUnpool2d(kernel_size=2)

pil = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")

fig, ax = plt.subplots(ncols=5, figsize=(20, 5), sharex=True, sharey=True)

ax[0].set_title("original " + str(pil.size))
ax[0].imshow(pil)
tensor = TF.to_tensor(pil).unsqueeze(0)
print("Orginal shape", tensor.shape)

# Downsample
tensor_half_res, indexes1 = pool(tensor)
tensor_show(tensor_half_res, "1/2 down ", ax=ax[1])

tensor_q_res, indexes2 = pool(tensor_half_res)
tensor_show(tensor_q_res, "1/4 down ", ax=ax[2])
print("Downsample shape", indexes2.shape)

# Upsample
tensor_half_res1 = unpool(tensor_q_res, indexes2)
tensor_show(tensor_half_res1, "1/2 up ", ax=ax[3])


tensor_recovered = unpool(tensor_half_res1, indexes1)
tensor_show(tensor_recovered, "full size up ", ax=ax[4])
print("Upsample shape", tensor_recovered.shape)
plt.show()

#### Transposed convolution

Способы восстановления пространственных размерностей, которые мы рассмотрели, не содержали обучаемых параметров. 

Для повышения пространственного разрешения карты признаков можно использовать операцию *Transposed convolution*, в которой, как в обычной свертке, есть **обучаемые параметры**. Альтернативное название: *Fractionally strided convolution*.

Иногда **некорректно** называется *обратной сверткой* или *Deconvolution*.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/simple_convolution.png" width="700"></center>
<center><em>Обычная свертка</em></center>

Операция обычной свертки накладывает фильтр-ядро на фрагмент карты, выполняет поэлементное умножение, а затем сложение, превращая **один фрагмент** входа в **один пиксель** выхода.

Transposed convolution, наоборот, проходит по всем пикселям входа и умножает их на **обучаемое ядро** свертки. При этом каждый **одиночный пиксель** превращается в **фрагмент**. Там, где фрагменты накладываются друг на друга, значения попиксельно суммируются.

Если вход имеет несколько каналов, то Transposed convolution применяет отдельный обучаемый фильтр к каждому каналу, а результат суммирует.

Параметр `stride` отвечает за дополнительный сдвиг каждого фрагмента на выходе. Используя Transposed convolution с параметром `stride = 2`, можно повышать размер карты признаков приблизительно в два раза, добавляя на нее мелкие детали.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/transposed_convolution_explained.png" width="1024"></center>

<center><em>Transposed convolution</em></center>

В отличие от обычной свертки, параметр `padding` Transposed convolution отвечает не за увеличение исходной карты признаков, а, наоборот, за "срезание" внешнего края карты-выхода. Это может быть полезно, потому что карта строится с перекрытием фрагментов, полученных из соседних пикселей, но по периметру результат формируется без перекрытия и может иметь более низкое качество.

Как правило, размер ядра `kernel_size` выбирают кратным `stride`, чтобы избавиться от артефактов при частичном наложении фрагментов, например:
```
kernel_size = 4
stride = 2
```
При таких значениях имеет смысл установить `padding=2`, чтобы убрать внешние два пикселя со всех сторон выходной карты признаков, полученные без перекрытия.

[Блог-пост про 2d свертки с помощью перемножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)

[Откуда слово Transposed в названии (раздел 4.1)](https://arxiv.org/pdf/1603.07285v1.pdf)

[Документация к ConvTranspose2d](
https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html?highlight=transpose#convtranspose2d)

```
torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, 
                         stride=1, padding=0, ...)
```
где:
* `in_channels`, `out_channels` — количество каналов в входной и выходной карте признаков,
* `kernel_size` — размер ядра свертки Transpose convolution, 
* `stride` — шаг свертки Transpose convolution,
* `padding`— размер отступов, устанавливаемых по краям входной карты признаков.



Пример использования:

In [ ]:
input = torch.randn(1, 16, 16, 16)  # define dummy input
print("Original size", input.shape)

downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1)  # define downsample layer
upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1)  # define upsample layer

# let`s downsample and upsample input
with torch.no_grad():
    output_1 = downsample(input)
    print("Downsampled size", output_1.size())

    output_2 = upsample(output_1, output_size=input.size())
    print("Upsampled size", output_2.size())

# plot results
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)
sns.heatmap(input[0, 0, :, :], ax=ax[0], cbar=False, vmin=-2, vmax=2)
ax[0].set_title("Input")
sns.heatmap(output_1[0, 0, :, :], ax=ax[1], cbar=False, vmin=-2, vmax=2)
ax[1].set_title("Downsampled")
sns.heatmap(output_2[0, 0, :, :], ax=ax[2], cbar=False, vmin=-2, vmax=2)
ax[2].set_title("Upsampled")
plt.show()

## Пирамида признаков

Возникает вопрос: не потеряется ли информация о мелких деталях изображения при передаче через центральный блок сети, где пространственное разрешение минимально? Такая проблема существует. 

Те, кто изучал классические методы машинного зрения, помнят, что  при извлечении дескрипторов особых точек([SIFT](https://en.wikipedia.org/wiki/Scale-invariant_feature_transform)) использовалась так называемая пирамида изображений.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/pyramid_of_features.png" width="650">

Идея состоит в последовательном уменьшении (масштабировании) изображения и последовательном извлечении признаков в разных разрешениях.

При уменьшении пространственных размеров мы естественным образом получаем карты признаков с разным пространственным разрешением.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_information.png" width="650">

Их можно использовать одновременно как вход для новых сверток так и для получения предсказаний.

На этой модели построены FPN - сети: 
[Feature Pyramid Networks for Object Detection](https://arxiv.org/abs/1612.03144).

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/fcn_1.png" width="1000">

<center><p><em>Source: <a href="https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1"> Review: FCN — Fully Convolutional Network (Semantic Segmentation) </a></p> </em></center>

После того как все карты признаком будут увеличены до одного размера они суммируются.

Примеры использования: [1](https://pytorch.org/hub/pytorch_vision_fcn_resnet101/) и [2](https://pytorch.org/vision/stable/models.html#semantic-segmentation)


Предобученная модель была обучена на части датасета COCO train2017 (на 20 категориях, представленных так же в датасете  Pascal VOC). Использовались следующие классы:

`['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']`

In [ ]:
import torchvision
from torchvision import transforms


def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


# load resnet50
fcn_model = torchvision.models.segmentation.fcn_resnet50(
    weights="FCN_ResNet50_Weights.DEFAULT", num_classes=21
)

classes = [
    "__background__",
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "diningtable",
    "dog",
    "horse",
    "motorbike",
    "person",
    "pottedplant",
    "sheep",
    "sofa",
    "train",
    "tvmonitor",
]

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # ImageNet
    ]
)

pil_img = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
input_tensor = transform(pil_img)

with torch.no_grad():
    output = fcn_model(input_tensor.unsqueeze(0))

Возвращаются 2 массива

* out — каждый пиксель отражает ненормированную вероятность, соответствующую предсказанию каждого класса.


In [ ]:
print("output keys: ", output.keys())  # Ordered dictionary
print("out: ", output["out"].shape, "Batch, class_num, h, w")

output_predictions = output["out"][0].argmax(0)  # for first element of batch
print(f"output_predictions: {output_predictions.shape}")

fig = plt.figure(figsize=(10, 10))
plt.imshow(pil_img)
plt.axis("off")
plt.show()

indexes = output_predictions
semantic_seg_person_predict = np.zeros(pil_img.size).astype(bool)

# plot all classes predictions
fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(10, 10))
i = 0  # counter
for row in range(4):
    for col in range(5):
        mask = torch.zeros(indexes.shape)
        mask[indexes == i] = 255

        ax[row, col].set_title(classes[i])
        ax[row, col].imshow(mask)
        ax[row, col].axis("off")
        i += 1

plt.show()

In [ ]:
semantic_seg_person_predict = torch.zeros(indexes.shape)
semantic_seg_person_predict[indexes == 15] = 1  # to obtain binary mask
semantic_seg_person_predict = (
    semantic_seg_person_predict.numpy()
)  # for skliarn compability

plt.imshow(semantic_seg_person_predict, cmap="gray")
plt.show()

## IoU — оценка точности

Как оценить качество предсказаний, полученных от модели?

Базовой метрикой является Intersection over Union (IoU), она же коэффициент Жаккара([Jaccard index](https://en.wikipedia.org/wiki/Jaccard_index))

Имеется предсказание модели (фиолетовая маска) и целевая разметка,  сделанная человеком (красная маска)*.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/iou_sample.png" width="400">

<left><p><em>Source: <a href="https://datahacker.rs/deep-learning-intersection-over-union/">Intersection over Union</a></p> </em></center>

Необходимо оценить качество предсказания.

**Для простоты в примере маски прямоугольные, но та же логика будет работать  для масок произвольной формы.*

Метрика считается как отношение площади пересечения к площади объединения двух масок: 

$$ IoU = \frac{|T \cap P|}{|T \cup P|} $$

T — True mask, P — predicted mask.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/iou_formula.png" width="400"></center>

<center><p><em>Source: <a href="https://towardsdatascience.com/non-maximum-suppression-nms-93ce178e177c"> Non-maximum Suppression (NMS)</a></p></em></center>

Если маски совпадут на 100%, то значение метрики будет равно 1 и это наилучший результат. При пустом пересечении IoU будет нулевым. Значения метрики лежат в интервале [0..1].

В терминах ошибок первого/второго рода IoU можно записать как:

$$ IoU = \frac{TP}{TP + FP + FN} $$


TP — True positive — пересечение (обозначено желтым),

FP — False Positive (остаток фиолетового прямоугольника),

FN — False Negative (остаток красного прямоугольника).

На базе этой метрики строится ряд производных от нее метрик, таких как Mean Average Precision, которую мы рассмотрим в разделе Детектирование.

Дополнительная информация: [Intersection over Union](http://datahacker.rs/deep-learning-intersection-over-union/)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))


plt.subplot(1, 3, 1)
plt.axis("off")
plt.title("GT mask")
plt.imshow(semantic_seg_person_mask)

plt.subplot(1, 3, 2)
plt.axis("off")
plt.title("Predicted mask")
plt.imshow(semantic_seg_person_predict)

plt.subplot(1, 3, 3)
plt.title("GT & Predict overlap")
plt.axis("off")
tmp = semantic_seg_person_predict * 2 + semantic_seg_person_mask
plt.imshow(tmp)
plt.show()

SKLearn: 
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html

SMP (func): https://smp.readthedocs.io/en/latest/metrics.html#segmentation_models_pytorch.metrics.functional.iou_score

Torchmetrics(obj): https://torchmetrics.readthedocs.io/en/stable/classification/jaccard_index.html

Torchvision (bbox): https://pytorch.org/vision/main/generated/torchvision.ops.box_iou.html



In [ ]:
from sklearn.metrics import jaccard_score

# wait vectors, so we flatten the data
y_true = semantic_seg_person_mask.flatten()
y_pred = semantic_seg_person_predict.flatten()
iou = jaccard_score(y_true, y_pred)

print(f"IoU = {iou:.2f}")

In [ ]:
from sklearn.metrics import confusion_matrix

# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
# or use:  smp.metrics.get_stats
# https://smp.readthedocs.io/en/latest/metrics.html#segmentation_models_pytorch.metrics.functional.get_stats
tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
iou = tp / (tp + fp + fn)
print(f"IoU = {iou:.2f}")

## Loss функции для сегментации


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/loss_overview.png" width="900"></center>

<center><a href="https://github.com/JunMa11/SegLoss">Loss functions for image segmentation</a></center>

### Distribution - based loss

Так как задача сегментации свобится к задаче классификации, то, можно использовать Cross-Entropy Loss, BCE, или Focal Loss которыми мы знакомы.



#### Binary Cross-Entropy (BCE) 

Если предсказывается маска для объектов единственного класса(target.shape = 1xHxW), то задача сводится к бинарной классификации. Так как каждый канал на выходе последнего слоя выдает предсказание для единственного класса.

Это позволяет заменить Softmax в Cross-Entropy Loss на сигмоиду, а функцию потерь — на бинарную кросс-энтропию (BCE).

In [ ]:
import torch

mask_class_1 = torch.randint(0, 2, (1, 64, 64))  # [0 , 1]
one_class_out = torch.randn(1, 1, 64, 64)
print(mask_class_1.shape)
print(one_class_out.shape)

Применяем [BCEWithLogitsLoss](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html) 

In [ ]:
from torch import nn

bce_loss_wl = nn.BCEWithLogitsLoss()  # Sigmoid inside
loss = bce_loss_wl(
    one_class_out, mask_class_1.float().unsqueeze(0)
)  # both params must have equal size
print(loss)

  Если последний слой модели это Сигмоида, то можем использовать [BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html)

In [ ]:
norm_one_class_out = one_class_out.sigmoid()

bce_loss = nn.BCELoss()
loss = bce_loss(
    norm_one_class_out, mask_class_1.float().unsqueeze(0)
)  # both params must have equal size
print(loss)

[CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) для одного класса работать не будет:

In [ ]:
cross_entropy = nn.CrossEntropyLoss()
loss = cross_entropy(one_class_out, mask_class_1.float().unsqueeze(0))
print(loss)

Так как softmax от единственного фхода всегда 1-ца

In [ ]:
one_class_out.softmax(dim=1).unique()

**Multilabel**
Если предсказывается несколько классов и target имеет форму: NxWxH (multilabel)
То есть маска каждого храниться в отдельном канале:


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/semantic_mask2.png" width="700">

[An overview of semantic image segmentation](https://www.jeremyjordan.me/semantic-segmentation/)



In [ ]:
import torch

mask_class1 = torch.randint(0, 2, (1, 64, 64))  # [0 , 1]
mask_class2 = torch.randint(0, 2, (1, 64, 64))

target = torch.cat((mask_class1, mask_class2))

print(target.shape)

То форма выхода модели совпадает с формой тензора масок:

In [ ]:
two_class_out = torch.randn(1, 2, 64, 64)
print(two_class_out.shape)

Мы можем посчитать [BCE](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html) поэлементно , предварительно преобразовав target во float

In [ ]:
from torch import nn

# https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html

bce_loss = nn.BCEWithLogitsLoss()  # Sigmoid inside
float_target = target.float()  # add batch and convert ot float
loss = bce_loss(
    two_class_out, float_target.unsqueeze(0)
)  # both params must have equal size
print(loss)

Или [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [ ]:
cross_entropy = nn.CrossEntropyLoss()
# If containing class probabilities, same shape as the input and each value should be between [0,1][0,1].
loss = cross_entropy(two_class_out, float_target.unsqueeze(0))
print(loss)

Результаты не совпадут так как после Sigmoid и Softmax получаться разные вероятности:



#### Cross-Entropy 

Если маска задана одним каналом, в котором классы пронумерованны целыми числами (multiclass) 
<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/semantic_mask1.png" width="900">

[An overview of semantic image segmentation](https://www.jeremyjordan.me/semantic-segmentation/)

In [ ]:
sq_target = target.argmax(0)
sq_target.shape

То логично использовать [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

```
Input: Shape (C), (N,C) or (N,C,d1​,d2​,...,dK​) with K≥1 in the case of K-dimensional loss.

Target: If containing class indices, shape (), (N) or (N,d1,d2,...,dK)
```


In [ ]:
# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
cross_entropy = nn.CrossEntropyLoss()

loss = cross_entropy(two_class_out, sq_target.unsqueeze(0))
print(loss)

### Region based loss

Лосс функции основанны на оценки площади пересечения масок.




#### Jaccard Loss

В отличие от accuracy, рассчет IoU (Jaccard index):
$IoU = JaccardIndex = \dfrac{  TP  }{TP + FP + FN} \in [0,1]$

можно произвести [дифференцируемым образом](https://arxiv.org/abs/1908.03851).

И тогда метрику можно превратить в лосс- функцию инвертировав ее:

$Jaccard Loss = 1 - IoU$




В Pytorch эта лосс функция не реализована, поэтому для ее использования установим библиотеку [SMP](https://github.com/qubvel/segmentation_models.pytorch) 

In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
import segmentation_models_pytorch as smp

iou_loss = smp.losses.JaccardLoss(smp.losses.MULTILABEL_MODE, from_logits=True)
print("IoU Loss", iou_loss(two_class_out, float_target.unsqueeze(0)))

\*при наличии на выходе модели сигмоиды или другой функции преобразующей логиты в вероятности, параметр `from_logits` следует уствновить равным `False`

#### Dice loss

Другой популярной метрикой для оценки качества сегментации является Dice коэффициент:

$ Dice =  \dfrac{  2 | AB | }{ |A|+|B| } $

Концептульно он похож на IoU но при выржении через ошибки первого и второго рода то будет видно что он совпадет F1 - мерой:

$Dice =  \dfrac{  2AB  } {A+B } = \dfrac{  2TP  }{2TP + FP + FN} = F1\_score ∈ [0,1]$

[F1 Score = Dice Coefficient](https://chenriang.me/f1-equal-dice-coefficient.html)



Его вычисление не сложно произвести дифференцируемым образом:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/dice_loss.jpeg" width="800">

$Dice = \dfrac{2\sum (y_{true}y_{pred})}{ \sum y_{true} + \sum y_{pred}}$

И затем превратить в лосс функцию: 

$ DiceLoss = 1 - Dice $



В PyTorch эти лосс функции не реализованы, поэтому установим библиотеку [SMP](https://github.com/qubvel/segmentation_models.pytorch) 

In [ ]:
dice_loss = smp.losses.DiceLoss(smp.losses.MULTILABEL_MODE, from_logits=True)

print(two_class_out.shape, target.shape)
print("DICE Loss", dice_loss(two_class_out, float_target.unsqueeze(0)))

Концептуально DiceLoss vs JaccardLoss похожи. Но JaccardLoss [сильнее штрафует можель на выбросах](https://stats.stackexchange.com/questions/273537/f1-dice-score-vs-iou).






Хороший обзор loss функции для семантической сегментации можно найти в репозитории: [Loss functions for image segmentation](https://github.com/JunMa11/SegLoss)

## U-Net: Convolutional Networks for Biomedical Image Segmentation

Сохраним большую часть традиционной структуры сети, включая слои downsampling (снижение пространственных размеров), извлечем основные признаки, а затем восстановим пространственные размеры.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/efficient_way_predict_pixel_class.png" width="800">

### Автокодировщик

Эта архитектура повторяет архитектуру автокодировщика (autoencoder)

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/encoder_loss.png" width="500">

Такая архитектура довольно популярна и применяется не только для сегментации, но и в следующих задачах: 

- сглаживание шума;
- снижение размерности $\to$ вектор-признак;
- генерация данных.

Этому будет посвящена одна из следующих лекций, сейчас же мы детально рассмотрим разжимающий блок.

Признаки полученные при сжатии скопируем и передадим в разжимающие слои, где карты признаков будут иметь соответствующее пространственное разрешение:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/add_skip_connection.png" width="650">

Так же, как и ResNet, этот механизм носит название skip connection, но  признаки  не суммируются, а конкатенируются.

Рассмотренная нами схема используется в U-Net. Эта популярная модель для сегментации медицинских изображений изначально была предложена в статье [U-Net: Convolutional Networks for Biomedical Image Segmentation (Ronneberger et al., 2015)](https://arxiv.org/abs/1505.04597) для анализа  медицинских изображений.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/unet_scheme.png" width="700"></center>
<center><em>Архитектура U-Net (Ronneberger et al., 2015).</em></center>

[Реализация на PyTorch](https://github.com/milesial/Pytorch-UNet)

[U-Net на PyTorch Hub](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/)

[Блог-пост разбор](https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5)

Стоит обратить особое внимание на серые стрелки на схеме: они соответствуют операции конкатенации копий ранее полученных карт активаций по аналогии с DenseNet. Чтобы это было возможно, необходимо поддерживать соответствие между размерами карт активаций в процессах снижения и повышения пространственных размерностей. Для этой цели изменения размеров происходят только при операциях `MaxPool` и `MaxUnpool` — в обоих случаях в два раза. 

В коде прямой проход может быть реализован, например, вот так:

```
def forward(self, x):
    out1 = self.block1(x) #  ------------------------------>
    out_pool1 = self.pool1(out1)

    out2 = self.block2(out_pool1)
    out_pool2 = self.pool2(out2)

    out3 = self.block3(out_pool2)
    out_pool3 = self.pool2(out3)

    out4 = self.block4(out_pool3)
    # return up
    out_up1 = self.up1(out4)

    out_cat1 = torch.cat((out_up1, out3), dim=1)
    out5 = self.block5(out_cat1)
    out_up2 = self.up2(out5)

    out_cat2 = torch.cat((out_up2, out2), dim=1)
    out6 = self.block6(out_cat2)
    out_up3 = self.up3(out6)

    out_cat3 = torch.cat((out_up3, out1), dim=1) # <-------
    out = self.block7(out_cat3)

    return out

```
После upsample блоков, ReLU не используется.

## Обзор DeepLabv3+ (2018)

DeepLab — семейство моделей для сегментации, значительно развивавшееся в течение четырёх лет. Основой данного рода моделей является использование **atrous (dilated) convolutions** и, начиная со второй модели, **atrous spatial pyramid pooling**, опирающейся на **spatial pyramid pooling**.

[Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation (Chen et al., 2018)](https://arxiv.org/abs/1802.02611v3)

[Реализация на PyTorch](https://pytorch.org/vision/stable/models.html#deeplabv3)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/deeplabv3_scheme.png" width="800">

### Atrous (Dilated) Convolution

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/dilated_convolution.png" width="650">

**Dilated convolution** (расширенная свертка) — это тип свертки, который "раздувает" ядро, как бы вставляя отверстия между элементами ядра. Дополнительный параметр (скорость расширения, **dilation**) указывает, насколько сильно расширяется ядро.


Фактически в такой свертке входные пиксели (признаки) участвуют через один (два, три ...).

Расширенные свертки позволяют значительно увеличить рецептивное поле и хорошо показывают себя при [решении задач семантической сегментации изображений](https://arxiv.org/pdf/1511.07122.pdf).


```
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, 
                padding=0, dilation=1, ...)
```
где:
* `in_channels`, `out_channels` — количество каналов в входной и выходной карте признаков,
* `kernel_size` — размер ядра свертки,
* `stride` — шаг свертки,
* `padding` — размер отступов, устанавливаемых по краям входной карты признаков,
* `dilation` — скорость расширения свертки.

[[doc] nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

#### <font color ="orange">доп инфо. про Atros convolution</font>

In [ ]:
# for plot
def plot_conv2d(input, conv, output, dilation=1):
    fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
    # input
    sns.heatmap(
        input[0][0],
        ax=ax[0],
        annot=True,
        fmt=".0f",
        cbar=False,
        vmin=0,
        vmax=20,
        linewidths=1,
    )
    # kernel
    sns.heatmap(
        conv.weight.detach()[0, 0, :, :],
        ax=ax[1],
        annot=True,
        fmt=".0f",
        cbar=False,
        vmin=0,
        vmax=20,
        linewidths=1,
    )
    # output
    sns.heatmap(
        output[0, 0, :, :],
        ax=ax[2],
        annot=True,
        fmt=".0f",
        cbar=False,
        vmin=0,
        vmax=20,
        linewidths=1,
    )
    # titles
    ax[0].set_title("Input \nshape: " + str(input.shape))
    ax[1].set_title("Kernel \nshape: " + str(conv.weight.shape))
    ax[2].set_title("Output \nshape: " + str(output.shape))
    fig.suptitle("Dilation = " + str(dilation), y=1.05)

    plt.show()

In [ ]:
# Atrous example
with torch.no_grad():
    # define dummy input
    # https://stackoverflow.com/questions/58584413/black-formatter-ignore-specific-multi-line-code
    # fmt: off
    input = torch.tensor([[[[1, 2, 3],
                            [1, 2, 3],
                            [1, 2, 3]]]], dtype=torch.float)
    # fmt: on
    # define conv layer, dilation = 1
    conv = nn.Conv2d(1, 1, kernel_size=2, dilation=1, bias=False)
    # define kernel weights
    conv.weight = nn.Parameter(torch.tensor([[[[1, 2], [2, 1]]]], dtype=torch.float))
    output = conv(input)
    plot_conv2d(input, conv, output, dilation=1)

In [ ]:
# change dilation to 2
with torch.no_grad():
    conv = nn.Conv2d(
        1, 1, kernel_size=2, dilation=2, bias=False
    )  # Fell free to change dilation
    conv.weight = nn.Parameter(torch.tensor([[[[1, 2], [2, 1]]]], dtype=torch.float))
    output = conv(input)
    plot_conv2d(input, conv, output, dilation=2)

In [ ]:
with torch.no_grad():
    input = torch.tensor([[[[0, 1, 0], [1, 1, 1], [0, 1, 0]]]], dtype=torch.float)
    output = conv(input)
    plot_conv2d(input, conv, output, dilation=2)

# Segmentation models PyTorch (SMP)

[Библиотека](https://github.com/qubvel/segmentation_models.pytorch#architectures-) на базе PyTorch  для сегментации.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/smp.png" width="1000">



In [ ]:
from IPython.display import clear_output


!pip install segmentation-models-pytorch

clear_output()

Можем комбинировать декодер с разными энкодерами

In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import torch

# 'mit_b0' Mix Vision Transformer Backbone from SegFormer pretrained on Imagenet
preprocess_input = get_preprocessing_fn("mit_b0", pretrained="imagenet")

# MixVisionTransformer encoder does not support in_channels setting other than 3
# supported by FPN only for encoder depth = 5
model = smp.FPN("mit_b0", in_channels=3, classes=10, encoder_depth=5)

# ... Train model on your dataset

dummy_input = torch.randn([1, 3, 64, 64])

mask = model(dummy_input)
print(mask.shape)  # torch.Size([1, 1, 64, 64])

### Совместимость с timm

Существует библиотека [pytorch-image-models](https://github.com/huggingface/pytorch-image-models) (timm = Torch IMage Models)


В которой собранно большое количество моделей для работы с изображениями

https://huggingface.co/docs/timm/index

https://huggingface.co/docs/hub/timm


In [ ]:
import timm

model_names = timm.list_models(pretrained=True)
print("Total pretrained models: ", len(model_names))

Можно искать модели по шаблону

In [ ]:
model_names = timm.list_models("*mobilenet*small*")
print(model_names)

Smoke test

In [ ]:
timm_mobilenet = timm.create_model("mobilenetv3_small_050", pretrained=True)

In [ ]:
out = timm_mobilenet(dummy_input)
print(out.shape)

### Совместимость с SMP

Можно использовать большинство моделей из timm в качестве энкодеров.

[Список совместимых моделей](https://smp.readthedocs.io/en/latest/encoders_timm.html).

При этом к названии модели которое передается в конструктор класса SMP нужно добавить перфикс `tu-`

In [ ]:
smp_timm_model = smp.DeepLabV3("tu-mobilenetv3_small_050", in_channels=3, classes=80)
smp_timm_model.eval()
print("Created DeepLab with mobileNet encoder")

Smoke test

In [ ]:
mask = smp_timm_model(dummy_input)
print(mask.shape)

# Детектирование (Object detection)


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/detection
.png" width="400">

Детектирование — задача компьютерного зрения, в которой требуется
определить местоположение конкретных объектов на изображении.

При этом вычислять точные границы объектов не требуется, а достаточно определить только ограничивающие прямоугольники (bounding boxes), в которых находятся объекты.

В общем случае объекты могут принадлежать к различным классам, и объектов одного класса на изображении может быть несколько. 

## Детектирование единственного объекта

Начнём с простой ситуации.

Пусть нас интересуют объекты только одного класса, и мы знаем, что такой объект на изображении есть и он один. 

К примеру, мы разрабатываем систему по распознаванию документов:

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/predict_bounded_box_example.png" width="700"></center>

<center><p><em>Source: <a href="https://arxiv.org/ftp/arxiv/papers/1807/1807.05786.pdf"> MIDV-500: a dataset for identity document analysis and recognition
on mobile devices in video stream</a></p> </em></center>

На вход модели подаётся изображение, и предсказать требуется область, в которой объект локализован.
Область (bounding box) определяется набором координат вершин*. Собственно эти координаты и должна предсказать модель.

\* *Если наложить условие, что стороны многоугольника должны быть параллельны сторонам изображения, то можно ограничиться предсказанием 2-х координат.*

### Предсказание координат

Если задачу семантической сегментации получилось свести к классификации, то здесь будем использовать регрессию, поскольку предсказывать нужно не номер класса, а набор чисел.

В зависимости от требований эти числа могут нести разный смысл, например:

* координаты центра + ширина и высота,
* координаты правого верхнего и левого нижнего углов,
* координаты вершин многоугольника ...

Но в любом случае задача остается регрессионной. 

Зафиксируем seeds для воспроизводимости результата.

In [ ]:
import torch
import random
import numpy as np


def fix_seeds():
    torch.manual_seed(42)
    random.seed(42)
    np.random.seed(42)


fix_seeds()

Решается она так:

Берем сверточную сеть и меняем последний полносвязный слой таким образом, чтобы количество выходов совпадало с количеством координат, которые нам нужно предсказать.

Так для предсказания двух точек потребуется четыре выхода ( x1 , y1 , x2, y2 )




In [ ]:
from torchvision.models import resnet18
from torch import nn

# load pretrained model
resnet_detector = resnet18(weights="ResNet18_Weights.DEFAULT")

# Change "head" to predict coordinates (x1,y1 x2,y2)
resnet_detector.fc = nn.Linear(resnet_detector.fc.in_features, 4)  # x1,y1 x2,y2

Для обучения такой модели придется заменить функцию потерь на регрессионную, например, MSE.

In [ ]:
criterion = nn.MSELoss()

# This is a random example. Don't expect good results
input = torch.rand((1, 3, 224, 224))
target = torch.tensor([[0.1, 0.1, 0.5, 0.5]])  # x1,y1 x2,y2 or x,y w,h
print(f"Target: {target}")
output = resnet_detector(input)
loss = criterion(output, target)
print(f"Output: {output}")
print(f"Loss: {loss}")

Координаты обычно предсказываются в процентах от длины и ширины изображения.
Таким образом, если bounding box целиком помещается на изображении, обе координаты будут находиться в интервале $[0 .. 1]$.




<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/predict_key_points.png" width="700"></center>
<center><em>Примеры предсказывания точек (Humphreys et al., 2020)</em></center>

[Recent Progress in Appearance-based Action Recognition (Humphreys et al., 2020)](https://arxiv.org/abs/2011.12619)


По такому принципу работают многие модели для поиска различных ключевых точек.
Например: на лице (facial landmarks) или теле человека.

### Multitask loss

Координаты прямоугольников мы предсказывать научились.

Теперь усложним задачу: объект остается один, но может принадлежать к различным классам.

 <img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/one_object.png" width="650">


То есть к задаче локализации добавляется классификация.

Задачу классификации мы умеем решать:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/class_prediction.png" width="650">

Остается объединить классификацию с регрессией: 

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/multitask_loss_0.png" width="650">

Для этого нужно одновременно предсказывать:

*   вероятность принадлежности к классам,
*   координаты ограничивающего прямогульника (bounding box).

Тогда выход последнего слоя будет иметь размер: 
$$N + 4$$

где N — количество классов (1000 для ImageNet), 

а 4 числа — это координаты одного boundig box (x1,y1,x2,y2 или x,y,w,h).


**Как описать функцию потерь для такой модели?**

Можно суммировать loss для классификации и loss для регрессии.

$ \large L_{total} = L_{crossentropy}+L_{mse}$

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/multitask_loss.png" width="650">

Однако значения разных loss могут иметь разные порядки, поэтому приходится подбирать весовые коэффициенты для каждого слагаемого. 
В общем случае функция потерь будет иметь вид:

$$\large L_{total} = \sum _iw_iL_i$$
где $w_i$ — весовые коэффициенты каждой из функций потерь. 

Они являются гиперпараметрами модели и требуют подбора.

### Подбор весов для каждой компоненты loss




Можно подбирать веса компонентов loss в процессе обучения. Для этого к модели добавляется дополнительный слой:

[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics(Alex Kendall et al., 2018)](https://arxiv.org/abs/1705.07115)


[Пример реализации MultiTask learning](https://github.com/Hui-Li/multi-task-learning-example-PyTorch/blob/master/multi-task-learning-example-PyTorch.ipynb)

## Детектирование нескольких объектов

Как быть, если объектов несколько?


Для каждого объекта нужно вернуть координаты(x1,y1,x2,y2) и класс (0 .. N). 
Соответственно, количество выходов модели надо увеличивать.

Но нам неизвестно заранее, сколько объектов будет на изображении:


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/object_detection_multiple_object.png" width="1000"> 

[Stanford University CS231n: Detection and Segmentation](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

**Cкользящее окно**
Одним из вариантов решения этой проблемы является применение классификатора ко всем возможным местоположениям объектов. Классификатор предсказывает, есть ли на выбранном фрагменте изображения один из интересующих нас объектов. Если нет, то фрагмент классифицируется как "фон". 

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/naive_way_object_detection_multiple_object.gif" width="700"></center>

<center><em>Source: <a href="http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf">Detection and Segmentation</a></em></center>




Проблемой данного подхода является необходимость применять классификатор к огромному количеству различных фрагментов, что крайне дорого с точки зрения вычислений.

## Object proposal

Чтобы сократить количество возможных окон для детекции, применяются разные способы:

1. На исходном изображении эвристический алгоритм предсказывает области, где скорее всего находятся объекты (~2K) (R-CNN).
2. Эти области предсказывает нейросеть (подсеть) (~2K) (Two stage detectors:  [Faster R-CNN](https://arxiv.org/abs/1506.01497)).
3. Эвристически задаются центры и размеры окон, которые плотно покрывают все изображение(~ 10K окон), (One stage detector: [SSD](https://arxiv.org/abs/1512.02325), YOLO).
4. Нейросеть предсказывает вероятность нахождения объекта и его размеры для каждой точки на карте признаков, так как на последних слоях карты признаков небольшие. (Anchor free: [FCOS](https://arxiv.org/abs/1904.01355))
5. Если в качестве кодировщика используется сеть на базе рхитектуры ViT, то предсказание делается для каждого patch (1K <) (DETR [link text](https://arxiv.org/abs/2005.12872)).


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/detector_types.png" width="1200">

**Общая схема работы детектора**:

* Изображение единожды пропускается через базовую часть сети (encoder),  дальнейшие предсказания делаются на карте признаков.
* Во всех случаях предсказываются смещение и масштабирование для финальных bounding box.
* Для каждого bounding box предсказывается класс.
* Предсказывается заведомо больше bounding box, чем может быть объектов на изображении.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/detector_scheme.png" width="1200">

Loss

Лосс складывается из лосс для классификации $L_{conf}$ и лосс для детекции $L_{loc}$

$$\large L_{det} = L_{conf} + \alpha L_{loc}$$

При этом в $L_{loc}$ учитываются **не все** предсказанные bounding box, а только те которые наилучшем образом пересекаются с GT (bbox из разметки). Фильтрация может проходить по порогу или при помощи алгоритма.

## NMS

Теперь возникает другая проблема: в районе объекта алгоритм генерирует множество ограничивающих прямоугольников (bounding box), которые частично перекрывают друг друга.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/non_max_suppression.png" width="650">

Чтобы избавиться от них, используется алгоритм
NMS (Non maxima suppression). Его задача — избавиться от bounding boxes, которые накладаваются на истинный:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/non_max_suppression_pseudo_code.png" width="1000">

<center><p><em>Source: <a href="https://towardsdatascience.com/non-maximum-suppression-nms-93ce178e177c"> Non-maximum Suppression (NMS)</a></p></em></center>

здесь $B$ — это массив всех bounding box,  $C$ — массив предсказаний модели относительно наличия объекта в соответствующем bounding box



Для оценки схожести обычно используется метрика IoU(same == IoU), а значение IoU ($\lambda_{nms}$), при котором bounding boxes считаются принадлежащими одному объекту, является гиперпараметром (часто 0.5).


В PyTorch алгоритм NMS доступен в модуле torchvision.ops

`torchvision.ops.nms(boxes, scores, iou_threshold)`,
где:
* `boxes` — массив bounding box,
* `scores` — предсказанная оценка,
* `iou_threshold` — порог IoU, NMS отбрасывает все перекрывающиеся поля с $IoU> iou\_threshold$

[[doc] torchvision.ops.nms](https://pytorch.org/vision/stable/generated/torchvision.ops.nms.html)

[Soft NMS](https://github.com/Gan4x4/ml_snippets/blob/main/CV/SoftNMS.ipynb)

## Backbone для детекторов

### Path Aggregation Network Module(PAN)



В backbone использующимися в детекторах на базе сверточных сетей применяются подходы развивающие идеи FPN


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/pan.png" width="600"></center>

<center><p>
<a href ="https://openaccess.thecvf.com/content_cvpr_2018/papers/Liu_Path_Aggregation_Network_CVPR_2018_paper.pdf" >[Path Aggregation Network for Instance Segmentation]</a>
</center></p>


а) Строится пирамида признаков, при этом некоторые карты признаков дополняются  признакими с более ранних слоев (красный пунктир)

b) Затем на основе последнего слоя FPN строится еще одна, (Bottom-up) и опять новые  карты признаков дополняются признакими полученными на первом уровне  (зеленый пунктир)

### Swin Transformer (2021)

[Aug 2021 Swin Transformer: Hierarchical Vision Transformer using Shifted Windows](https://arxiv.org/abs/2103.14030)


Применять ViT напрямую для задач сегментации и детектирования не слишком эффективно, так как при больших размерах patch (16x16) не получится получить точные границы объектов.

А при уменьшении размеров patch будет требоваться все больше ресурсов, так как сложность self-attention $O(n^{2})$ пропорциональна квадрату количества элементов на входе.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/swin_vs_vit.png" width="600"></center>

<center><em>Source: <a href="https://arxiv.org/pdf/2103.14030.pdf">Swin Transformer: Hierarchical Vision Transformer using Shifted Windows</a></em></center>

Авторы решают проблему при помощи двух усовершенствований.

Self-attention применяется не ко всему изображению сразу, а к его большим фрагментам, окнами.

На первый взгляд это возвращает проблему сверток, про которую мы говорили:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/cnn_fail.png" width="600">

Указываем сети, куда смотреть, и это помешает оценить взаимное влияние признаков расположенных на разных углах карты.

Чтобы не допустить этой проблемы, на каждом следующем transformer-слое окно сдвигается.



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/swin_window_shift.png" width="700">

<center><p><em>Source: <a href="https://arxiv.org/pdf/2103.14030.pdf">Swin Transformer: Hierarchical Vision Transformer using Shifted Windows</a></p> </em></center>



Таким образом сеть может выучить влияние любого патча на любой. При этом не требуется увеличивать количество входов self-attention блока, и количество вычислений не растет. 

Далее пространственные размеры карт признаков уменьшаются аналогично тому, как это происходит в сверточных сетях. Для сегментирования и детектирования используется принцип FPN: признаки с разных пространственных карт агрегируются для предсказания.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/swin_architecture.png" width="1200">

<center><p><em>Source: <a href="https://arxiv.org/pdf/2103.14030.pdf">Swin Transformer: Hierarchical Vision Transformer using Shifted Windows</a></p> </em></center>

Patch merging здесь — это конкатенация эмбеддингов с последующей подачей на вход линейного слоя.

Фрагмент из 4-х эмбеддингов 2x2xC конкатенируются. Получаем один тензор 1x1x4C.

Затем подаем его на вход линейному слою,  уменьшающему число каналов в 2 раза,
получаем новый эмбеддинг размерностью 1x1x2C.

Таким образом, в отличие от традиционных трансформер-архитектур, размер embedding здесь меняется.


<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/swin_result.png" width="500"></center>

<center><em>Source: <a href="https://arxiv.org/pdf/2103.14030.pdf">Swin Transformer: Hierarchical Vision Transformer using Shifted Windows</a></em></center>

Такой подход позволил достичь SOTA результатов как в задаче классификации, так и в задачах детектирования и сегментации. 
Авторы статьи позиционируют Swin Transformer как backbon решения широкого круга задач CV.

## YOLO

* [You Only Look Once: Unified, Real-Time Object Detection (Redmon et. al., 2015)](https://arxiv.org/abs/1506.02640) 
* [YOLO9000: Better, Faster, Stronger (Redmon et. al., 2015)](https://arxiv.org/abs/1612.08242)
* [YOLOv3: An Incremental Improvement (Redmon et. al., 2018)](https://arxiv.org/abs/1804.02767)
* [YOLOv4: Optimal Speed and Accuracy of Object Detection (Bochkovskiy et al., 2020)](https://arxiv.org/abs/2004.10934)
* [YOLOv5 (Glenn Jocher Ultralytics,  June 2020)](https://github.com/ultralytics/yolov5)
* [YOLOX: Exceeding YOLO Series in 2021 (Ge et al., June 2021)](https://arxiv.org/abs/2107.08430)

* [YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object detectors(Chien-Yao Wang et. al. July 2022)](https://arxiv.org/abs/2207.02696)

*  [YOLOv6: A Single-Stage Object Detection Framework for Industrial Applications(Chuyi Li et. al., Sept 2022)](https://arxiv.org/abs/2209.02976)

*  	[YOLOv8,	Ultralytics, Dec. 2022](
https://github.com/ultralytics/ultralytics)
[(documentation !)](https://docs.ultralytics.com/)

* [YOLO6v3	YOLOv6 v3.0: A Full-Scale Reloading Chuyi Li et. al., Dec. 2023](https://arxiv.org/abs/2301.05586)


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolov6_performance.png" width="1000">

<center><p><em>Source: <a href="https://arxiv.org/abs/2301.05586">YOLOv6 v3.0: A Full-Scale Reloading
</a></p> </em></center>


Первая версия YOLO вышла в том же году что и SSD. На тот момент детектор несколько проигрывал SSD в точности.

Однако благодаря усилиям Joseph Redmon проект поддерживался и развивался в течение нескольких лет.

 3-я версия детектора оказалась настолько удачной, что даже в 2021 можно было прочесть:  "YOLOv3, one of the most widely used detectors in industry" [2021](https://arxiv.org/abs/2107.08430)

Последующие версии создавались разными авторами. Полагаю что правильно считать их разными форками YOLOv3 а не новыми версиями. Даже нумерация условна, например статья про v7 датируется более ранней датой чем v6.

В прикладных задачах я бы рекомендовал использовать YOLOv8 так как авторы выложили на свой сайт [документацию](https://docs.ultralytics.com/), а точность при скорости порядка 100 fps у всех современных моделей почти одинакова.

В настоящий момент можно сказать, что YOLO — это оптимальный детектор по соотношению качества распознавания к скорости.

Подробнее здесь: https://github.com/Gan4x4/ml_snippets/blob/main/Detectors_history.ipynb

Мы же запустим одну из последних моделей:


**YOLOv8**

По этой модели не публиковалась статья, зато есть [документация](https://docs.ultralytics.com/quickstart/)

Попробуем попробуем запустить ее.

In [ ]:
from IPython.display import clear_output


!pip install ultralytics

clear_output()

Инстанцируем модель по названию. Полный [список предобученных моделей](https://github.com/ultralytics/ultralytics#models) .

In [ ]:
from ultralytics import YOLO

yolo8 = YOLO("yolov8n.pt")

Детектируем объекты на изображении из COCO.

Из коробки работает с изображениями в разных форматах и даже url, автоматически меняет размер входного изображения, возвращает объект с результатами...

https://docs.ultralytics.com/modes/predict/#arguments

In [ ]:
results = yolo8.predict(
    "http://images.cocodataset.org/val2017/000000448263.jpg",  # baseball
    conf=0.25,  # for NMS
    iou=0.7,
)  # for NMS

В качестве результата возвращается список объектов которые содержат информацию полную информацию о детектировании.

https://docs.ultralytics.com/modes/predict

In [ ]:
print(len(results))  # contains detections for one image

У него есть методы для получения списка координат предсказанных bounding box после NMS

In [ ]:
# https://docs.ultralytics.com/reference/results/
print(results[0].boxes.data)  # x1,y2,x2,y2,conf,class_num

Может даже создать картинку с нарисоваными bounding box. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
pil_with_bbox = results[0].plot()
plt.imshow(pil_with_bbox)  # BGR?
plt.show()

Похоже объект results хранит картинку в BGR формате, переведем в RGB

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(pil_with_bbox[..., ::-1])  # BGR->RGB
plt.show()

# Instance Segmentation

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/detection_instance_segmentation.png" width="650">

**Mask R-CNN** (Detectron) — концептуально простая, гибкая и общая схема сегментации объектов. Подход эффективно обнаруживает объекты на изображении и одновременно генерирует высококачественную маску сегментации для каждого объекта. 

Метод, названный Mask R-CNN, расширяет Faster R-CNN (который мы обсуждали ранее), добавляя ветвь для предсказания маски объекта параллельно с существующей ветвью для распознавания *bounding boxes*. 

Код доступен [по ссылке](https://github.com/facebookresearch/Detectron)

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/mask_r_cnn.png" width="750"></center>

<center><em>Source: <a href="http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf"> Detection and Segmentation </a></em></center>

[Модель Mask R-CNN](https://pytorch.org/vision/stable/models.html#mask-r-cnn)

[Пример запуска Mask R-CNN есть в документации PyTorch](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)



Загрузим картинку

In [ ]:
from IPython.display import clear_output


!wget "http://images.cocodataset.org/val2017/000000448263.jpg"

clear_output()

In [ ]:
from PIL import Image
from torchvision.transforms import ToTensor

I = Image.open("000000448263.jpg")
t = ToTensor()(I)

Загрузим модель

In [ ]:
from torchvision.models.detection import (
    maskrcnn_resnet50_fpn,
    MaskRCNN_ResNet50_FPN_Weights,
)

mask_rcnn = maskrcnn_resnet50_fpn(weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT)
clear_output()
mask_rcnn.eval()
predictions = mask_rcnn(t.unsqueeze(0))

Для каждого изображения возвращается словарь

In [ ]:
print(predictions[0].keys())

Каждый ключ содержит тензор. Количество элементов в котором равно количеству детектированных объектов.

In [ ]:
predictions[0]["masks"].shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def show_masks(masks, classes=None):

    fig, ax = plt.subplots(nrows=4, ncols=8, figsize=(20, 10))
    i = 0
    for row in range(4):
        for col in range(8):
            if classes is not None:
                ax[row, col].set_title(int(classes[i]))
            ax[row, col].imshow(masks[i])
            ax[row, col].axis("off")
            i += 1
            if i >= len(masks):
                plt.show()
                return

    plt.show()


show_masks(
    predictions[0]["masks"].detach().squeeze(1), predictions[0]["labels"].detach()
)

Маски не бинарные, при необходимости можем их бинаризовать

In [ ]:
mask = predictions[0]["masks"][2].detach().squeeze(0)
mask = mask > 0.5
plt.imshow(mask, cmap="gray")
plt.show()

У YOLOv8 заявлена поддержка сегментации:

https://docs.ultralytics.com/tasks/segment/

In [ ]:
!pip install ultralytics

clear_output()

In [ ]:
from ultralytics import YOLO


yolo_seg = YOLO("yolov8m-seg.pt")

In [ ]:
seg_results = yolo_seg(I)

In [ ]:
seg_results[0].masks.shape

In [ ]:
seg_results[0].masks.data.shape

In [ ]:
seg_results[0].boxes.cls

In [ ]:
plt.figure(figsize=(10, 6))
pil_with_bbox = seg_results[0].plot(boxes=True, masks=True)
plt.imshow(pil_with_bbox[..., ::-1])  # BGR?
plt.show()

# Panoptic Segmentation

[COCO panoptic](https://cocodataset.org/#panoptic-2020)

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/panoptic_segmentation.png" width="700"></center>

<center><em>Source: <a href="https://kharshit.github.io/blog/2019/10/18/introduction-to-panoptic-segmentation-tutorial">Introduction to Panoptic Segmentation: A Tutorial</a></em></center>

#OWL-ViT (2022)




Vision Transformer for Open-World Localization

[Simple Open-Vocabulary Object Detection with Vision Transformers](https://arxiv.org/abs/2205.06230)

Код: https://huggingface.co/docs/transformers/model_doc/owlvit

Детектирование по произвольному текстовому запросу.

Модели на вход подается изображение и набор текстов. Модель предсказывает bbox , и сравнивает их эмбеддинги с эмбеддингами текстов.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/owl-vit.png" width="1000">

[Simple Open-Vocabulary Object Detection with Vision Transformers](https://arxiv.org/abs/2205.06230)




*  Лосс для bbox такой же как в [DETR](https://arxiv.org/abs/2005.12872) (`bipartite matching`)
*  Эмбеддинги учаться аналогично [CLIP](https://arxiv.org/abs/2103.00020)

In [ ]:
from IPython.display import clear_output


!pip install transformers

clear_output()

Создаем модель и `processor`.

`processor` — это класс, отвечающий за предобработку и постобработку данных.

In [ ]:
import torch
from transformers import OwlViTProcessor, OwlViTForObjectDetection

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = OwlViTProcessor.from_pretrained(
    "google/owlvit-base-patch32",
)
model = OwlViTForObjectDetection.from_pretrained(
    "google/owlvit-base-patch32",
)
model.eval()
model.to(device)
print("loaded")  # suppress huge output of model structure

Загрузим изображение

In [ ]:
!wget -N "http://images.cocodataset.org/val2017/000000448263.jpg"

clear_output()

In [ ]:
from PIL import Image


img = Image.open("000000448263.jpg")
img

Создадим набор текстовых описаний для поиска

In [ ]:
texts = ["cap", "botle", "text", "boy", "player"]  #

In [ ]:
batch = [img]
batch_size = len(batch)
with torch.inference_mode():
    inputs = processor(
        text=texts * batch_size,  # copy the same text for all images
        images=[img],  # batch of images in PIL format
        return_tensors="pt",
    )
    inputs = inputs.to(device)
    outputs = model(**inputs)  # return object of type OwlViTObjectDetectionOutput

In [ ]:
print("bboxex", outputs["pred_boxes"].shape)  # batch * total_boxes * 4 coords per box
print("logits", outputs["logits"].shape)  # batch * total_boxes * text_count

Постобработка

При предобработке изображения могут масштабироваться (resize), что бы получить корректные bbox а выходе, в метод можно подать список исходных размеров изображений.

In [ ]:
size = img.size[::-1]  # WH -> HW
target_sizes = torch.tensor([size])
print(target_sizes)

In [ ]:
results = processor.post_process_object_detection(
    threshold=0.1, outputs=outputs, target_sizes=target_sizes.to(device)  #
)

print(results)

In [ ]:
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import box_convert
import matplotlib.pyplot as plt
import numpy as np


def draw_bbox(img, bb, color, labels=None, xywh=True):
    t_img = torch.tensor(img).permute(2, 0, 1)  # to tensor CHW
    bb = bb[:, :4].clone().detach()  # take only coords
    if xywh:
        bb = box_convert(bb, "xywh", "xyxy")  # convert from COCO format
    img_with_bb = draw_bounding_boxes(t_img, bb, colors=color, width=2, labels=labels)
    return img_with_bb.permute(1, 2, 0).numpy()  # back to numpy HWC


def show_owl_results(results):
    labels = []
    for i, text_idx in enumerate(results[0]["labels"]):
        labels.append(texts[text_idx] + f" {results[0]['scores'][i]:.2f}")
    image_with_bb = draw_bbox(
        np.array(img), results[0]["boxes"], "lime", labels=labels, xywh=False
    )
    plt.figure(figsize=(8, 6))
    plt.imshow(image_with_bb)
    plt.show()


show_owl_results(results)

Что бы получть больше результатов меняем порог

In [ ]:
coarse_results = processor.post_process_object_detection(
    threshold=0.07,  # change threshold
    outputs=outputs,
    target_sizes=target_sizes.to(device),
)

show_owl_results(coarse_results)

[Ёще пример. ](https://colab.research.google.com/github/google-research/scenic/blob/main/scenic/projects/owl_vit/notebooks/OWL_ViT_minimal_example.ipynb)


#SAM (2023)

[Segment Anything](https://arxiv.org/abs/2304.02643)


Модель возвращает набор масок соответствующих входу. Классы объектов не используются.

В качестве входа может подаваться: 

*  набор точек
*  набор bounding box
*  маски
*  текст (подержка в коде пока не реализована)
*  изображение 


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/sam_overview.png" width="1000">

Обучалась на огромном датасете, частично размеченном в unsupervise режиме

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/sam_architecture.png" width="1000">

Установим пакет

In [ ]:
from IPython.display import clear_output


!pip install git+https://github.com/facebookresearch/segment-anything.git

clear_output()

Загрузка весов

https://github.com/facebookresearch/segment-anything#model-checkpoints

In [ ]:
# ViT-H
!wget -N https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

clear_output()

Создаем encoder

In [ ]:
import torch
from segment_anything import sam_model_registry  # SamPredictor, sam_model_registry

# model_type = 'vit_h'
sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h_4b8939.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam.to(device=device)
print("")

Загрузим изображение

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_1.png -O cat.png

In [ ]:
!wget "http://images.cocodataset.org/val2017/000000448263.jpg"

clear_output()

In [ ]:
from PIL import Image
import numpy as np

img = Image.open("000000448263.jpg")
np_im = np.array(img)  # HWC format
img

Предсказываем все маски

https://github.com/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb

Создадим эмбеддинг (на CPU выполняется долго) и предскажем все маски

In [ ]:
%%time
from segment_anything import SamAutomaticMaskGenerator

mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(np_im)

На выходе получаем список

In [ ]:
masks[0]

In [ ]:
masks[0]["segmentation"].shape

In [ ]:
# https://github.com/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb
import matplotlib.pyplot as plt


def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x["area"]), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones(
        (
            sorted_anns[0]["segmentation"].shape[0],
            sorted_anns[0]["segmentation"].shape[1],
            4,
        )
    )
    img[:, :, 3] = 0
    for ann in sorted_anns:
        m = ann["segmentation"]
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(img)
show_anns(masks)
plt.axis("off")
plt.show()

Предсказываем по точкам.
https://github.com/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb
Сначала создаем эмбеддинг. Он хранится внутри модели.

In [ ]:
%%time

from segment_anything import SamPredictor


predictor = SamPredictor(sam)
predictor.set_image(np_im)  # create embedding

Теперь получаем предсказания, указав точки, которые относятся к объекту и фону:

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=np.array([[200, 200], [1, 1]]),  # point coords
    point_labels=np.array([1, 0]),  # 1 - object(foreground), 0 - background
    # box
    # mask_input
    multimask_output=True,  # return 1 or 3 masks because of the ambiguous input
)

In [ ]:
print("Masks count", len(masks))
print("Scores", scores)

In [ ]:
print(masks[0].shape)

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


plt.imshow(img)
show_mask(masks[2], plt.gca())
plt.axis("off")
plt.show()

# Оценка качества детекции

## mAP — mean Average Precision

AP (Average Precision — средняя точность) — это площадь под сглаженной PR-кривой.

m (mean) —  усредненная для разных порогов IoU.

Разберемся, что это значит.

### Precision & recall

**Precision** измеряет, насколько точны предсказания сети (т.е. процент правильных предсказаний)

**Recall** измеряет, насколько хорошо сеть находит все положительные срабатывания (*positives*). Например, мы можем найти 80% возможных положительных срабатываний в наших K лучших предсказаниях.

Вот их математические определения:

$\displaystyle\mathrm{Precision} = \frac{TP}{TP+FP}$

$\displaystyle\mathrm{Recall} = \frac{TP}{TP+FN}$

где $TP$ — True Positive, $TN$ — True Negative, $FP$ — False Positive, $FN$ — False Negative.



Например, мы пытаемся детектировать яблоки на фотографиях. Предположим, мы обработали 20 фотографий (на 10 фотографиях по одному яблоку и на 10 фотографиях яблок нет) и обнаружили что:

* в 7 случаях наша нейросеть обнаружила яблоко там, где оно было на самом деле (True Positive);
* в 3 случаях не обнаружила яблоко там, где оно было (False Negative);
* в 4 случаях обнаружила яблоко там, где его не было (False Positive);
* в 6 случаях правильно определила, что на фотографии яблок нет (True Negative).



Посчитаем precision и recall:

In [ ]:
def precision(TP, FP):
    return TP / (TP + FP)


def recall(TP, FN):
    return TP / (TP + FN)


pres = precision(TP=7, FP=4)
rec = recall(TP=7, FN=3)

print("Precision = %.2f" % pres)
print("Recall = %.2f" % rec)

### PRCurve

Зная Precision и Recall для разных порогов, можно построить так называемую [PR-кривую](https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/).

[Как построить PR curve](https://github.com/Gan4x4/ml_snippets/blob/main/Training/PR_curve.ipynb).

При помощи таких кривых часто оценивают модели в задачах классификации, когда данные не сбалансированы.

Для получения значений Precision и Recall требуются:

*  Ground True метки;
*  уверенность модели в каждом предсказании.

In [ ]:
import numpy as np

y_true = np.array([1, 1, 0])  # Labels
y_pred = np.array([0.8, 0.1, 0.2])  # Predictions

Этого достаточно для того, чтобы построить PR-кривую для предсказаний одного класса.

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt


def pr_curve(y_true, y_pred):
    plt.figure(figsize=(5, 5))
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    plt.plot(recall, precision, marker="o")
    for i, t in enumerate(thresholds):
        plt.annotate(str(t), (recall[i], precision[i]))
    plt.ylim([0, 1.1])
    plt.xlabel("recall")
    plt.ylabel("precision")
    plt.title("PR curve")
    plt.show()


pr_curve(y_true, y_pred)

### IoU вместо метки класса


При детектировании у нас  **нет меток класса** для предсказанных bounding 
box.

Есть предсказанные детектором bbox в виде вида:

In [ ]:
import pandas as pd

detections = np.array(
    [
        [290, 50, 170, 160, 0.7, 53],  # x, y, w, h, confidence, class_num
        [10, 200, 190, 180, 0.8, 53],
        [310, 250, 120, 130, 0.75, 53],
    ]
)
pd.DataFrame(
    data=detections, columns=["x", "y", "width", "height", "confidence", "class_num"]
)


И есть Ground True bounding box из разметки:  

In [ ]:
from IPython.display import clear_output


# !wget -N "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
!wget -N "https://edunet.kea.su/repo/EduNet-web_dependencies/L12/annotations_trainval2017.zip"
!unzip -n annotations_trainval2017.zip

clear_output()

In [ ]:
from pycocotools.coco import COCO


coco = COCO("annotations/instances_val2017.json")
clear_output()

apples_img_id = 60855  # if of some image with apples
apple_cat_id = 53  # apple

ann_id = coco.getAnnIds(imgIds=[apples_img_id])
anns = coco.loadAnns(ann_id)

gt_bbox = []
for ann in anns:
    if ann["category_id"] == apple_cat_id:
        gt_bbox.append(ann["bbox"] + [ann["category_id"]])


pd.DataFrame(data=gt_bbox, columns=["x", "y", "width", "height", "class_num"])

Идея состоит в том, чтобы посчитать максимальный IoU между этими bounding box и использовать его вместо метки класса (True/False).



In [ ]:
import torch
import skimage
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import box_convert


def draw_bbox(img, bb_xywh, color):
    t_img = torch.tensor(img).permute(2, 0, 1)  # to tensor CHW
    xywh = torch.tensor(bb_xywh)[:, :4]  # take only coords
    bb = box_convert(xywh, "xywh", "xyxy")  # convert from COCO format
    img_with_bb = draw_bounding_boxes(t_img, bb, colors=color, width=4)
    return img_with_bb.permute(1, 2, 0).numpy()  # back to numpy HWC


img_info = coco.loadImgs(apples_img_id)

img = skimage.io.imread(img_info[0]["coco_url"])
img = draw_bbox(img, detections, "blue")
img = draw_bbox(img, gt_bbox, "lime")
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.show()

Чтобы получить максимальное значение IoU для каждого предсказанного bbox,  воспользуемся функцией [torchvision.ops.box_iou](https://pytorch.org/vision/main/generated/torchvision.ops.box_iou.html).

Перед этим преобразуем формат координат к `(x1,y1,x2,y2)`:

In [ ]:
from torchvision.ops import box_convert

gt = box_convert(torch.tensor(gt_bbox)[:, :4], "xywh", "xyxy")
pred = box_convert(torch.tensor(detections)[:, :4], "xywh", "xyxy")

И получим IoU для всех возможных пар bbox:


In [ ]:
from torchvision.ops import box_iou

pairwise_iou = box_iou(gt, pred)
print(pairwise_iou)

Нас интересует только максимальный:

In [ ]:
iou, _ = pairwise_iou.max(dim=0)
print(iou)

Запишем эти значения в таблицу.


In [ ]:
import pandas as pd

predictions = torch.tensor(detections)[:, 4]
predictions = torch.vstack((predictions, iou)).T

pd.DataFrame(data=predictions, columns=["confidence", "iou"])

Тперь будем использовать IoU как GT метку класса: если он больше некоторго порога, то предсказание верное.

В нашем случае будем считать, что если IoU≥0.5, то предсказание правильное.

In [ ]:
gt = iou > 0.5

predictions = torch.vstack((predictions[:, 0], gt)).T
detection_results = pd.DataFrame(data=predictions, columns=["confidence", "gt"])
detection_results.head()

Теперь у нас есть все необходимое для того, чтобы построить PR-кривую:

In [ ]:
from sklearn.metrics import precision_recall_curve

plt.figure(figsize=(7, 5))
precision, recall, thresholds = precision_recall_curve(gt, predictions[:, 0])
plt.plot(recall, precision, marker="o")
plt.ylim([0, 1.1])
plt.show()

### Average Precision

AP это площадь под PR кривой.

$\large \displaystyle AP = \int_0^1p(r)dr$

Имея значения Precision и Recall, можно посчитать AP/

In [ ]:
from sklearn.metrics import average_precision_score

ap = average_precision_score(gt, predictions[:, 0])
print("AP", ap)

Сравним это значение с площадью под кривой:

In [ ]:
from sklearn import metrics

auc = metrics.auc(recall, precision)
print("AUC", auc)

Как видим, они не совпали. Дело в том, что при подсчете AP кривую сглаживают:

In [ ]:
import numpy as np


def smooth_precision(precision):
    smooth_prec = []
    for i in range(1, len(precision) + 1):
        max = precision[:i].max()
        smooth_prec.append(max)
    return smooth_prec


detection_results = detection_results.sort_values("confidence", ascending=False)
precision, recall, thresholds = precision_recall_curve(
    detection_results["gt"], detection_results["confidence"]
)

smoothed_precision = smooth_precision(precision)
plt.figure(figsize=(7, 5))
plt.plot(recall, precision, marker="o")
plt.plot(recall, smoothed_precision, marker="o")
plt.ylim([0, 1.1])
plt.show()

In [ ]:
smooth_auc = metrics.auc(recall, smoothed_precision)
print("smooth AUC ", smooth_auc)

### COCO mAP

Мы посчитали AP для одного класса, но в датасете их много.
Логично усреднить значения AP для разных классов. В некоторых бенчмарках после такого усреднения к названию метрики добавляют приставку m(mean)

*AP is averaged over all categories. Traditionally, this is called “mean average precision” (mAP). We make no distinction between AP and mAP*

### AP для разных порогов.

Мы выбрали для IoU порог 0.5.

Но можно посчитать AP для разных порогов, и затем тоже усреднить. В этом случае в названии метрики указывается минимальный и максимальный пороги. 
Например AP@[.5:.95] соответствует среднему AP для IoU от 0.5 до 0.95 с шагом 0.05. 

In [ ]:
!pip install torchmetrics

clear_output()

In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision

t_gt_bbox = torch.tensor(gt_bbox)
t_detections = torch.tensor(detections)

map_obj = MeanAveragePrecision("xywh", iou_thresholds=[0.5, 0.55, 0.6, 0.65])
results = map_obj(
    preds=[
        {
            "boxes": t_detections[:, :4],  # xywh
            "scores": t_detections[:, 4],  # confidence
            "labels": t_detections[:, 5],  # class num
        }
    ],
    target=[
        {"boxes": t_gt_bbox[:, :4], "labels": t_gt_bbox[:, 4]}  # xywh  # class num
    ],
)

print("mAP@[0.5:0.65] = ", results["map"].item())

# Практические соображения

* Сегментация: если данных мало и они специфические, используйте UNet. 
* Если данных достаточно, используйте DeepLab в качестве baseline.
* Используйте пакеты SMP и torchmetrics.
* Для детектирования используйте YOLOv8 или YOLOv6.
* Если ваша задача связана с трекингом, обратитесь к преподавателю.



<font size ="6">Список использованной литературы</font>

<font size ="5">COCO</font>

[Подробнее о том, как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

[Подробнее о разметке COCO](https://cocodataset.org/#format-data)

[Что такое  run-length encoding - RLE](https://en.wikipedia.org/wiki/Run-length_encoding)?

[Видео-разбор Run-length encoding](https://www.youtube.com/watch?v=h6s61a_pqfM)

<font size ="5">Семантическая сегментация</font>

[FCN Semantic segmenation](https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1)

[Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation (Chen et al., 2018)](https://arxiv.org/abs/1802.02611v3)

[Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition (He et al., 2014)](https://arxiv.org/abs/1406.4729)

[Блог-пост про семантическую сегментацию](https://www.jeremyjordan.me/semantic-segmentation/)

<font size ="5">Детектирование</font>

[Recent Progress in Appearance-based Action Recognition (Humphreys et al., 2020)](https://arxiv.org/abs/2011.12619)

[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

[SSD: Single Shot MultiBox Detector (Liu et al., 2015)](https://arxiv.org/abs/1512.02325)

[Focal Loss for Dense Object Detection (Lin et al., 2017)](https://arxiv.org/abs/1708.02002)

[Статья про Focal Loss от Facebook AI Research](https://arxiv.org/abs/1708.02002)

[Feature Pyramid Networks for Object Detection (Sergelius et al., 2016)](https://arxiv.org/abs/1612.0314)

[Understanding feature pyramid networks for object detection - FPN](https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)

[You Only Look Once: Unified, Real-Time Object Detection (Redmon et. al., 2015)](https://arxiv.org/abs/1506.02640) 

[YOLO9000: Better, Faster, Stronger (Redmon et. al., 2015)](https://arxiv.org/abs/1612.08242)

[YOLOv3: An Incremental Improvement (Redmon et. al., 2018)](https://arxiv.org/abs/1804.02767)

[YOLOv4: Optimal Speed and Accuracy of Object Detection (Bochkovskiy et al., 2020)](https://arxiv.org/abs/2004.10934)

[YOLOv5 (Glenn Jocher)](https://github.com/ultralytics/yolov5)

[YOLOX: Exceeding YOLO Series in 2021 (Ge et al., 2021)](https://arxiv.org/abs/2107.08430)

<font size ="5">Hard Example Mining</font>

[Training Region-based Object Detectors with Online Hard Example Mining (Shrivastava et al., 2016)](https://arxiv.org/abs/1604.03540)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors (Yu et al., 2018)](https://arxiv.org/abs/1804.04606)

<font size ="5">DINO</font>

[Emerging Properties in Self-Supervised Vision Transformers (Caron et al., 2021)](https://arxiv.org/abs/2104.14294)

[Отличное видео объяснение статьи DINO](https://www.youtube.com/watch?v=h3ij3F3cPIk)

<font size ="5">Другое</font>

[U-Net: Convolutional Networks for Biomedical Image Segmentation (Ronneberger et al., 2015)](https://arxiv.org/abs/1505.04597)

[Блог-пост про 2d свертки с помощью перемножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)